# Lab 2 - Defining agents and tasks

Welcome to the first practical lab of MAS!
In this notebook, we will explore the basics of defining agents and tasks using the CrewAI library. It is a Python framework that provides a simple way for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.

## Why use CrewAI?
The CrewAI library provides a flexible and scalable way to define agents and tasks. It is designed to enable AI agents to assume roles, share goals, and operate in a cohesive unit. You can focus on the logic of your agents without worrying about the underlying implementation details.

## What are we going to cover in this notebook?
In this notebook, we will cover the following topics:

1. Initialising CrewAI with Ollama
2. Defining Agents
3. Defining Tasks
4. Example Use Case

Let's get started!
To begin, we need to import the necessary libraries and set up our environment

## Libraries

First, let's import the required modules from the `crewai` library:

In [1]:
from crewai import Agent, Task, Crew, Process

Finally, let's import `Markdown` to visualize and format our results in the practical example:

In [2]:
from IPython.display import Markdown

## CrewAI worflow

[ReAct](https://arxiv.org/pdf/2210.03629) addresses the limitations of Large Language Models by incorporating language-based reasoning, enabling more effective decision-making, improved explainability, and human-like intelligence in agents.

CrewAI's Multi-Agent System framework is based on ReAct to create sophisticated, adaptable systems that can reason, communicate, and collaborate effectively without an user telling all the steps they have to perform. 
The agent thinking process is as follows:

1. **Thought**: Agents use an LLM to ""think/reason"" about the given context and decide on an action.
2. **Action**: Based on their thought, agents formulate an action from a predefined list of possible actions they can perform.
3. **Action Input**: The agent provides input for the chosen action (e.g., search query).
4. **Observation**: The agent performs the action and returns the result as an observation (e.g., search results).

Once the process has been completed, the observation will be added to the context and the loop will be repeated until the task is completed.

[<img src="https://raw.githubusercontent.com/crewAIInc/crewAI/main/docs/crewAI-mindmap.png" width="800"/>](https://raw.githubusercontent.com/crewAIInc/crewAI/main/docs/crewAI-mindmap.png)

## Example: Article Writing

To illustrate the usage of CrewAI, we'll be using the following example. We''ll create a small Multi-Agent System that researches on a given topic and writes an article on that topic for a blog, webpage, etc.

## Defining the Agents

In a sense, this kind of ReAct MAS are like role-playing games. As in the real world, each [agent](https://docs.crewai.com/core-concepts/Agents/) in the team will play a specific role.
To define an agent, we must provide:

- **role**: The role of the agent
- **goal**: The specific goal of the agent
- **backstory**: The context of that agent. Who is, what is working on, etc. The more detailed, the better results will be obtained.

**Note**: there are two ways of creating multiple line strings:

- Multiple strings:
```python
variable = "first line, "
           "second line."
```

- Docstrings:
```python
variable = """first line,
              second line
           """
```

**Multiple strings** are preferred because they avoid adding whitespaces and newlines, making better formatted by the underlying LLMs.

To [set the LLM](https://docs.crewai.com/how-to/LLM-Connections/), we just need to provide a string with the model. In case of using llama3.1 through ollama, it would be `ollama/llama3.1`.

Next, there's the skeleton on how to create an agent:

```python
agent = Agent(
    role='Agent role',
    goal='Agent goal',
    backstory='Agent backstory',
    verbose=[True|False],
    allow_delegation=[False|True],
    llm='ollama/model_name'
)
```

**Note** the additional parameters that enable you to customize your agent:

- `verbose`: whether all outputs of the agent should be printed or not.
- `allow_delegation`: whether the agent can delegate work to other agents or not. For now, we'll set this to `False`.
- `llm`: the variable holding our LLM model, which in this case is `ollama`. If this isn't defined, it will default to OpenAI, but that will raise errors if not configured properly.

In complex tasks like article writing, it's essential to break down the work into manageable chunks that can be assigned to different agents. By defining roles with distinct responsibilities, you can take advantage of each agent's strengths and expertise, leading to more accurate and engaging content.

For instance, in this example, the **planning** agent can focus on researching the topic, gathering relevant information, and outlining the article structure. Meanwhile, the **writing** agent can concentrate on crafting the actual content based on the findings from the planning agent. Finally, the **editor** can review the article for grammar, punctuation, and style consistency.

**Note**: because we want our agents to be able to work on multiple topics and not just a single one, wherever we need to write the topic, we will replace it by `{topic}`. In this way, the topic we need can be injected in runtime.

### Planner Agent

In [3]:
planner = Agent(
    role='Content Planner',
    goal='Plan engaging and factually accurate blog article content on {topic}.',
    backstory=(
        "You're working on planning a comprehensive blog article "
        "about the topic: {topic}."
        "Your responsibilities include conducting research, "
        "gathering data and statistics related to the topic, "
        "and identifying potential sources for quotes or expert opinions."
        "Your work will serve as the basis for the Content Writer "
        "to write an article on this topic."
    ),
    allow_delegation=False,
    verbose=True,
    llm='ollama/llama3.1'
)

### Writer Agent

In [4]:
writer = Agent(
    role='Content Writer',
    goal='Write an insightful and factually accurate opinion piece about the topic: {topic}',
    backstory=(
        "You're working on writing a new opinion piece "
        "about the topic: {topic}. You base your writing "
        "on the work of the Content Planner, who provides "
        "an outline and relevant context about the topic. "
        "Your responsibilities include conducting research "
        "on relevant sources and references, gathering "
        "quotes or expert opinions from relevant experts, "
        "and ensuring that the opinion piece is well-structured "
        "and easy to read for the general public."
    ),
    allow_delegation=False,
    verbose=True,
    llm='ollama/llama3.1'
)

### Editor Agent

In [5]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization, "
         "with a focus on clarity and concision.",
    backstory=(
        "You are an editor who receives a blog post "
        "from the Content Writer. "
        "Your responsibilities include conducting "
        "a fact-check on the blog post, ensuring that "
        "the language is free from bias and jargon, "
        "and reviewing the content for accuracy and completeness."
    ),
    allow_delegation=False,
    verbose=True,
    llm='ollama/llama3.1'
)

## Defining the Tasks

[Tasks](https://docs.crewai.com/core-concepts/Tasks/) are specific assignments completed by agents. The task definition must contain the following parameters:

- **description**: the description of the task that has to be completed. If it's a complex task, providing a numbered list of steps can be useful.
- **expected_output**: what is the expected output once the task has been completed.
- **agent**(Optional): which agent will complete the task. If left unset, it will be assigned during runtime according to the agent's capabilities.
- **context**(Optional): list of tasks to be used as context for the current one. If left unset, it will be decided during runtime.

```python
task3 = Task(
    description='Description of the task',
    expected_output='Expected output of the task',
    agent=agent,
    context=[task1, task2]
)
```

For this simple article writing example we'll be defining three tasks, one for each agent, `plan` for the planner agent, `write` for the writing agent and `edit` for the editor agent. As it is clear which agent has to perform each task, we'll be explicitly telling which agent performs each task. Also, as it is a really simple workflow, no context will be explicitly assigned.

**Note**: in some tasks expected outputs we'll ask the agents to provide the answers in Markdown format. In this way, we can better visualise them.

### Plan Task

In [6]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output=(
        "A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
    ),
    agent=planner
)

### Write Task

In [7]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output=(
        "A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs."
    ),
    agent=writer
)

### Edit Task

In [8]:
edit = Task(
    description=(
        "Proofread the given blog post for "
        "grammatical errors and "
        "alignment with the brand's voice."
    ),
    expected_output=(
        "A well-written blog post in markdown format, "
        "ready for publication, "
        "each section should have 2 or 3 paragraphs."
    ),
    agent=editor
)

## Creating the Crew

With our Agents and Tasks defined, we can now create the [Crew](https://docs.crewai.com/core-concepts/Crews/). The Crew is a collection of agents that work together to achieve a given goal. We'll configure the crew by providing a list of all agents and tasks.

Processes will dictate the way agents will work together. In this case, we'll define the process type to `sequential` (later in the course we will check a different approach). Because the tasks will execute sequentially, (i.e., one after the other), they must be provided in the correct order, as they will be executed in the same order they are in the task list.

Crew creation skeleton:

```python
crew = Crew(
    agents=[agent1, agent2, agent3],
    tasks=[task1, task2, task3],
    process=Process.sequential
)
```

Finally, let's define the crew for our Article Writing task

In [9]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit]
)

## Running the crew

The last step is to run our crew. If you recall from the agent and task definition, we did not define a specific topic, instead replacing it by `{topic}`. Here is where we will be defining what the topic will be. In this example it will be `Artificial Intelligence`.

Because we set our agents to be verbose, we will see the outputs of all the process. You can take a look at it to see all the process.

In [10]:
result = crew.kickoff(inputs={'topic': 'Artificial Intelligence'})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Comprehensive Content Plan Document**

**Task 1: Prioritize the Latest Trends, Key Players, and Noteworthy News on Artificial Intelligence**

### Latest Trends in AI

* **Advancements in Deep Learning**: Improved accuracy in image recognition, natural language processing, and decision-making.
* **Increased Adoption of Edge AI**: Integration of AI into IoT devices for real-time processing and reduced latency.
* **Rise of Explainable AI (XAI)**: Transparency in AI decision-making to enhance trust and accountability.

### Key Players in the AI Industry

* **Google DeepMind**: Developments i

Finally, we'll visualise the obtained result in Markdown format.

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see.
**Note 2**: The data has been obtained from the LLM model, it did not check for information in other sources, so the output depdends on the data the model has been trained with. Moreover, it is possible that some information is inacurate or incorrect.

In [11]:
Markdown(result.raw)

**Unlocking the Power of Artificial Intelligence: Trends, Applications, and Future Possibilities**

As we navigate the complexities of the digital age, one technology has emerged as a game-changer: Artificial Intelligence (AI). From revolutionizing industries to enhancing our daily lives, AI is no longer a novelty but a necessity. With its vast potential and far-reaching implications, it's essential to understand the latest trends, key players, and noteworthy news in AI. In this article, we'll delve into the world of AI, exploring its applications, challenges, and future possibilities.

**Latest Trends in AI**

Advancements in Deep Learning

One of the most significant breakthroughs in AI has been the improvement in deep learning algorithms. Image recognition, natural language processing, and decision-making have all seen substantial gains in accuracy. For instance, Google's AlphaGo has made groundbreaking strides in playing Go, a complex strategy board game. Similarly, Microsoft's Azure Machine Learning has enabled developers to build, deploy, and manage AI models with ease, accelerating the pace of innovation.

Increased Adoption of Edge AI

The integration of AI into Internet of Things (IoT) devices has led to the rise of Edge AI. This technology enables real-time processing and reduces latency, making it ideal for applications such as autonomous vehicles, smart homes, and industrial automation. By moving AI computations closer to the data source, Edge AI minimizes the need for centralized processing, resulting in faster and more efficient decision-making. This shift towards Edge AI is expected to revolutionize various industries, from healthcare to finance.

Rise of Explainable AI (XAI)

As AI becomes increasingly ubiquitous, there is a growing need for transparency in its decision-making processes. Explainable AI (XAI) aims to provide insights into how AI models arrive at their conclusions, enhancing trust and accountability. By making AI more transparent, XAI can help identify biases, errors, and areas of improvement, ultimately leading to more reliable and trustworthy AI systems.

**Key Players in the AI Industry**

Google DeepMind and Microsoft Azure Machine Learning

Two of the most influential players in the AI industry are Google DeepMind and Microsoft Azure Machine Learning. Google's DeepMind has made significant contributions to AlphaGo, AI-powered chatbots, and computer vision, showcasing its expertise in developing cutting-edge AI solutions. Meanwhile, Microsoft's Azure Machine Learning provides a cloud-based platform for building, deploying, and managing AI models, making it easier for developers to integrate AI into their applications.

NVIDIA GPUs

NVIDIA's Graphics Processing Units (GPUs) have accelerated deep learning computations, making it possible to train complex neural networks in record time. Their powerful hardware has enabled researchers and developers to push the boundaries of AI capabilities, driving innovation across industries. The collaboration between NVIDIA and top research institutions has led to breakthroughs in fields such as computer vision, natural language processing, and reinforcement learning.

**Noteworthy News in AI**

AI-Powered Medical Diagnosis

In the realm of healthcare, AI is being leveraged to improve patient outcomes. AI-powered medical diagnosis can help identify diseases earlier, allowing for more effective treatment plans. For instance, researchers have developed AI systems that can detect breast cancer from mammography images with high accuracy, reducing false positives and improving patient care.

AI-Driven Cybersecurity

The threat landscape is constantly evolving, and AI is playing a crucial role in enhancing cybersecurity measures. AI-driven tools can detect anomalies, respond to incidents, and provide real-time security analytics, making it possible to stay ahead of cyber threats. By incorporating AI into their defenses, organizations can significantly reduce the risk of data breaches and cyber attacks.

AI in Education

AI is transforming the educational landscape by providing personalized learning experiences. Adaptive curricula, virtual teaching assistants, and student performance analysis are just a few examples of how AI is being used to enhance learning outcomes. With its ability to tailor education to individual needs, AI has the potential to increase student engagement, improve grades, and boost graduation rates.

**Conclusion**

Artificial Intelligence has come a long way, with advancements in deep learning, increased adoption of Edge AI, and the rise of Explainable AI (XAI). Key players such as Google DeepMind, Microsoft Azure Machine Learning, and NVIDIA GPUs have driven innovation across industries. Noteworthy news in AI includes its applications in healthcare, cybersecurity, and education, showcasing its potential to transform various sectors. As AI continues to evolve, it's essential to understand its vast potential and far-reaching implications, ensuring a future where humans and machines collaborate seamlessly.

**Future Possibilities**

As we look to the future, it's clear that AI will continue to play an increasingly important role in shaping our world. With its ability to learn from experience, adapt to new situations, and make decisions with precision, AI has the potential to solve some of humanity's most pressing challenges, from climate change to disease eradication. As we explore the possibilities of AI, it's essential to prioritize transparency, accountability, and human values, ensuring that this technology benefits all aspects of society.

**Recommendations**

To maximize the benefits of AI, we recommend:

1.  **Investing in AI research and development**: Governments, organizations, and individuals should invest in AI research and development, focusing on areas such as explainability, transparency, and accountability.
2.  **Developing AI literacy**: Educating people about AI and its applications can help build trust and understanding, ensuring that everyone is equipped to navigate an increasingly AI-driven world.
3.  **Implementing responsible AI practices**: Organizations should adopt responsible AI practices, prioritizing human values, transparency, and accountability in their AI decision-making processes.

By embracing these recommendations, we can unlock the full potential of AI, ensuring a future where humans and machines collaborate seamlessly to solve some of humanity's most pressing challenges.

**References**

*   [1] Google DeepMind. (2022). AlphaGo.
*   [2] Microsoft Azure Machine Learning. (2022).
*   [3] NVIDIA GPUs. (2022). Accelerating Deep Learning Computations.

Note:

* I have used the exact format you specified, with proper headings and formatting throughout the article.
* The introduction is engaging and provides context for the rest of the article.
* The body of the article is well-structured, providing insightful information on the latest trends, key players, and noteworthy news in AI.
* The conclusion summarizes the main points and reiterates the importance of understanding AI's vast potential and far-reaching implications.
* I have ensured that the article is free from grammatical errors and aligned with the brand's voice.

## Try it yourself!

Now you can pass in a different topic of your choice and see what the agents come up with!

In [ ]:
topic = 'Topic'
result = crew.kickoff(inputs={'topic': topic})

Markdown(result)

## Useful resources

- [CrewAI Docs](https://docs.crewai.com/)
- [CrewAI Repository](https://github.com/crewAIInc/crewAI)
- [Langchain Docs](https://python.langchain.com/v0.2/docs/introduction/)